In [6]:
import torch
import random

def create_random_conflict_matrix(n):
    # Initialize a 24x24 matrix with zeros
    matrix = torch.zeros(n, n)

    # Randomly mark some adjacent seats as conflicting (1)
    for i in range(n):
        if i > 0 and random.random() < 0.5:
            matrix[i][i - 1] = 1
            matrix[i - 1][i] = 1
        if i < n - 1 and random.random() < 0.5:
            matrix[i][i + 1] = 1
            matrix[i + 1][i] = 1

    return matrix

# Create 500 random conflict matrices
num_iterations = 500
random_conflict_matrices = [create_random_conflict_matrix(24) for _ in range(num_iterations)]

#List to tensor
conflict_matrices = torch.stack(random_conflict_matrices, dim=0)

In [7]:

def calculate_total_conflicts(arrangement, conflict_matrix):
    """
    Calculates the total conflicts for a given arrangement based on the conflict matrix.
    Args:
        arrangement (torch.Tensor): Binary tensor representing the arrangement (1 for occupied, 0 for vacant).
        conflict_matrix (torch.Tensor): Conflict matrix (1 indicates conflict, 0 indicates no conflict).
    Returns:
        int: Total number of conflicts.
    """
    # Ensure that the arrangement and conflict_matrix have the same shape
    assert arrangement.shape == conflict_matrix.shape, "Shapes of arrangement and conflict_matrix must match."

    # Compute the element-wise product of arrangement and conflict_matrix
    conflicts = arrangement * conflict_matrix

    # Sum the conflicts
    total_conflicts = torch.sum(conflicts)

    return total_conflicts.item()


In [8]:
def custom_loss(predictions, conflict_matrix):
    """
    Custom loss function that penalizes adjacent seats being occupied simultaneously.
    Args:
        predictions (torch.Tensor): Predicted values (e.g., occupancy probabilities).
        conflict_matrix (torch.Tensor): Conflict matrix (1 indicates conflict, 0 indicates no conflict).
    Returns:
        torch.Tensor: Loss value.
    """
    # Compute the conflict term (sum of conflicts)
    conflict_term = torch.sum(predictions * conflict_matrix)

    # Compute the mean squared error (MSE) term
    mse_term = torch.mean(predictions ** 2)

    # Combine the terms (you can adjust the weights as needed)
    loss = mse_term + 0.1 * conflict_term

    return loss

In [11]:
import torch
import torch.nn as nn
import torch.optim as optim

class OccupancyNet(nn.Module):
    def __init__(self, input_size):
        super(OccupancyNet, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)  # Hidden layer with 64 neurons
        self.fc2 = nn.Linear(64, 24*24)  # Output layer (24x24 neurons for occupancy prediction)

    def forward(self, x):
        x = torch.relu(self.fc1(x))  # Apply ReLU activation to hidden layer
        x = self.fc2(x)  # Output layer (no activation function)
        return x.view(-1, 24, 24)  # Reshape to 24x24 grid

# An Example of 24 for seats
input_size = 24  # Number of seats (features)
model = OccupancyNet(input_size)

# Generate random input data (replace with your actual data)
batch_size = 500
input_data = torch.rand(batch_size, input_size)

# Forward pass through the model
prediction= model(input_data)


In [12]:
# the seating order for the first 24 people
seat1=torch.argmax(prediction[0], dim=0)
print(seat1.reshape((6, 4)))

tensor([[ 8, 18, 17,  9],
        [21, 13,  5,  1],
        [ 0,  0, 20,  9],
        [19, 17, 11, 15],
        [11,  1, 13, 20],
        [15, 21, 21, 13]])


In [ ]:
######################################################################################################################################